In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
db = DFDB('../trial/extratrees.pkl', auto_commit=False)

In [9]:
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])

In [11]:
df_trial[['datetime','nfeatures', 'kfold', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].loc[0:19]

,datetime,nfeatures,kfold,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-10 08:42:07.651275,1071,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.499165e-06,1.468636e-14,2.269274,0.489602,2.269272,None
1,2019-05-10 09:02:37.071088,200,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.197445e-06,9.774471e-15,2.260614,0.457957,2.260612,None
2,2019-05-10 09:16:09.476415,190,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.155385e-06,1.348192e-14,2.255028,0.468715,2.255027,None
3,2019-05-10 09:28:58.440117,180,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.146699e-06,5.994585e-15,2.240094,0.479207,2.240093,None
4,2019-05-10 09:41:04.616695,170,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.162662e-06,6.818891e-15,2.225713,0.467771,2.225712,None
5,2019-05-10 09:52:28.280265,160,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.145904e-06,1.329546e-14,2.214935,0.484027,2.214934,None
6,2019-05-10 10:03:09.978947,150,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.146723e-06,1.277012e-14,2.209481,0.474300,2.209479,None
7,2019-05-10 10:13:08.307155,140,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.156019e-06,1.259827e-14,2.203670,0.460615,2.203669,None
8,2019-05-10 10:22:23.215791,130,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.120365e-06,7.589212e-15,2.204208,0.458124,2.204207,None
9,2019-05-10 10:30:54.595849,120,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.104875e-06,1.124369e-14,2.209440,0.455868,2.209439,None


In [12]:
param_idx = 19
column_idx = 1
db_ = db
df_trial_ = df_trial
mytrial = []
columns = copy.deepcopy(df_trial_.loc[column_idx]['param']['columns'])
param = copy.deepcopy(df_trial_.loc[param_idx]['param'])
# param['algorithm']['init'] = {'logging_level': 'Silent', **param['algorithm']['init']}
param['scaler']['init'] = {}
param['algorithm']['fit'] = {}

selected_columns = EP.revert_rfe(df_train, param, columns, df_test, mytrial, start_columns=[columns[0]], limit=20, remark='start from top1 column')
print(len(selected_columns))
for trial_i in mytrial:
    db_.insert(trial_i)
df_trial_ = db_.select()
df_trial_['kfold'] = df_trial_['param'].apply(lambda x: x['kfold'])

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted t

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 1

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was conver

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted to float64 by StandardScaler.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning:

Data with input dtype int64 was converted t

...

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning:

The default value of n_estimators wil

2


In [13]:
db_.commit()

In [21]:
mytrial =[]
columns_ = selected_columns

#  tune hypterparameters
def objective(trial):
        
    n_estimators = trial.suggest_int('n_estimators', 300, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 16)
    max_features = trial.suggest_uniform('max_features', .6, 1)
    min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0.1, 0.5)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':columns_,
        'kfold':{
            'n_splits': 8,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'ExtraTreesRegressor',
            'init':{
                "n_estimators":n_estimators,
                "max_depth":max_depth,
                "max_features":max_features,
                "min_samples_leaf":min_samples_leaf,
                "random_state":random_state,
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 31')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-18 02:19:36,736] Finished trial#0 resulted in value: 0.0050894843376959. Current best value is 0.0050894843376959 with parameters: {'n_estimators': 382, 'max_depth': 7, 'max_features': 0.6079740494623934, 'min_samples_leaf': 0.3054629525707597, 'random_state': 1571}.
[I 2019-05-18 02:19:44,300] Finished trial#1 resulted in value: 0.007106717702670673. Current best value is 0.0050894843376959 with parameters: {'n_estimators': 382, 'max_depth': 7, 'max_features': 0.6079740494623934, 'min_samples_leaf': 0.3054629525707597, 'random_state': 1571}.
[I 2019-05-18 02:19:58,776] Finished trial#2 resulted in value: 0.028383279745097436. Current best value is 0.0050894843376959 with parameters: {'n_estimators': 382, 'max_depth': 7, 'max_features': 0.6079740494623934, 'min_samples_leaf': 0.3054629525707597, 'random_state': 1571}.
[I 2019-05-18 02:20:02,613] Finished trial#3 resulted in value: 0.004284617257594717. Current best value is 0.004284617257594717 with parameters: {'n_estimator

[I 2019-05-18 02:26:08,736] Finished trial#56 resulted in value: 0.000853535534643246. Current best value is 0.0001544224933720231 with parameters: {'n_estimators': 363, 'max_depth': 8, 'max_features': 0.7769834125962415, 'min_samples_leaf': 0.42641174786288655, 'random_state': 4293}.
[I 2019-05-18 02:26:16,212] Finished trial#57 resulted in value: 0.0018789830344064918. Current best value is 0.0001544224933720231 with parameters: {'n_estimators': 363, 'max_depth': 8, 'max_features': 0.7769834125962415, 'min_samples_leaf': 0.42641174786288655, 'random_state': 4293}.
[I 2019-05-18 02:26:19,479] Finished trial#58 resulted in value: 0.0053917356346006905. Current best value is 0.0001544224933720231 with parameters: {'n_estimators': 363, 'max_depth': 8, 'max_features': 0.7769834125962415, 'min_samples_leaf': 0.42641174786288655, 'random_state': 4293}.
[I 2019-05-18 02:26:26,054] Finished trial#59 resulted in value: 0.002155478982627481. Current best value is 0.0001544224933720231 with para

[I 2019-05-18 02:33:36,895] Finished trial#112 resulted in value: 0.009001230640000888. Current best value is 6.68917978773872e-05 with parameters: {'n_estimators': 738, 'max_depth': 10, 'max_features': 0.655682696220891, 'min_samples_leaf': 0.4514812547578928, 'random_state': 1513}.
[I 2019-05-18 02:33:42,437] Finished trial#113 resulted in value: 0.007108296159053132. Current best value is 6.68917978773872e-05 with parameters: {'n_estimators': 738, 'max_depth': 10, 'max_features': 0.655682696220891, 'min_samples_leaf': 0.4514812547578928, 'random_state': 1513}.
[I 2019-05-18 02:33:47,156] Finished trial#114 resulted in value: 0.0025267240918934435. Current best value is 6.68917978773872e-05 with parameters: {'n_estimators': 738, 'max_depth': 10, 'max_features': 0.655682696220891, 'min_samples_leaf': 0.4514812547578928, 'random_state': 1513}.
[I 2019-05-18 02:33:54,444] Finished trial#115 resulted in value: 0.0025316222234748956. Current best value is 6.68917978773872e-05 with paramet

[I 2019-05-18 02:40:49,710] Finished trial#168 resulted in value: 0.0013210301418392506. Current best value is 5.057993529946858e-05 with parameters: {'n_estimators': 746, 'max_depth': 8, 'max_features': 0.6591715323995949, 'min_samples_leaf': 0.4403434459042438, 'random_state': 1823}.
[I 2019-05-18 02:40:58,373] Finished trial#169 resulted in value: 0.00691379834715369. Current best value is 5.057993529946858e-05 with parameters: {'n_estimators': 746, 'max_depth': 8, 'max_features': 0.6591715323995949, 'min_samples_leaf': 0.4403434459042438, 'random_state': 1823}.
[I 2019-05-18 02:41:05,936] Finished trial#170 resulted in value: 0.0057052114108454835. Current best value is 5.057993529946858e-05 with parameters: {'n_estimators': 746, 'max_depth': 8, 'max_features': 0.6591715323995949, 'min_samples_leaf': 0.4403434459042438, 'random_state': 1823}.
[I 2019-05-18 02:41:13,447] Finished trial#171 resulted in value: 0.0017990232455131299. Current best value is 5.057993529946858e-05 with par

In [22]:
for trial_i in mytrial:
    db.insert(trial_i)

In [26]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 31')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
38,2019-05-18 02:19:58.774681,tune 31,26,2.087228,0.002689,2.100740,0.011794,0.013511
106,2019-05-18 02:28:02.968220,tune 31,26,2.096824,0.002917,2.107731,0.011098,0.010907
170,2019-05-18 02:36:55.845698,tune 31,26,2.096855,0.002657,2.108490,0.011098,0.011635
168,2019-05-18 02:36:31.553186,tune 31,26,2.097166,0.002743,2.108900,0.011653,0.011734
70,2019-05-18 02:23:49.624177,tune 31,26,2.104083,0.002617,2.114081,0.010877,0.009998
139,2019-05-18 02:32:35.815365,tune 31,26,2.107934,0.002582,2.117755,0.010808,0.009821
52,2019-05-18 02:21:56.562468,tune 31,26,2.112076,0.002707,2.121235,0.010151,0.009158
234,2019-05-18 02:44:44.799108,tune 31,26,2.121468,0.002395,2.128497,0.010967,0.007029
44,2019-05-18 02:20:51.276682,tune 31,26,2.127635,0.002389,2.133555,0.010270,0.005920
86,2019-05-18 02:25:35.054788,tune 31,26,2.128958,0.002528,2.135937,0.010452,0.006979


In [27]:
db.commit()

In [28]:
param = copy.deepcopy(df_trial.loc[38]['param'])
param['kfold']['type'] = 'stratified'
# run one try
mytrial=[]
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='remodel 38 use stratified')

In [29]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [30]:
df_trial.tail(1)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
236,2019-05-18 06:42:38.618614,26,2.090315,0.000003,2.091928,0.000026,0.001613


In [31]:
db.commit()

In [32]:
mytrial =[]
columns_ = copy.deepcopy(df_trial.loc[31]['param']['columns'])

#  tune hypterparameters
def objective(trial):
        
    n_estimators = trial.suggest_int('n_estimators', 300, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 16)
    max_features = trial.suggest_uniform('max_features', .6, 1)
    min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0.1, 0.5)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':columns_,
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'ExtraTreesRegressor',
            'init':{
                "n_estimators":n_estimators,
                "max_depth":max_depth,
                "max_features":max_features,
                "min_samples_leaf":min_samples_leaf,
                "random_state":random_state,
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 31 by stratified')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-18 06:52:53,878] Finished trial#0 resulted in value: 0.0013919887895316116. Current best value is 0.0013919887895316116 with parameters: {'n_estimators': 490, 'max_depth': 5, 'max_features': 0.7588264461176745, 'min_samples_leaf': 0.368778750076565, 'random_state': 8428}.
[I 2019-05-18 06:52:59,718] Finished trial#1 resulted in value: 0.00015975684167758775. Current best value is 0.00015975684167758775 with parameters: {'n_estimators': 634, 'max_depth': 5, 'max_features': 0.6531683699354258, 'min_samples_leaf': 0.461450941631973, 'random_state': 8955}.
[I 2019-05-18 06:53:14,716] Finished trial#2 resulted in value: 0.001468141648769292. Current best value is 0.00015975684167758775 with parameters: {'n_estimators': 634, 'max_depth': 5, 'max_features': 0.6531683699354258, 'min_samples_leaf': 0.461450941631973, 'random_state': 8955}.
[I 2019-05-18 06:53:23,075] Finished trial#3 resulted in value: 0.0023335043589044475. Current best value is 0.00015975684167758775 with parameter

[I 2019-05-18 07:01:57,414] Finished trial#56 resulted in value: 0.00011602541979556042. Current best value is 6.240043953774098e-06 with parameters: {'n_estimators': 904, 'max_depth': 8, 'max_features': 0.9905947830545193, 'min_samples_leaf': 0.4999748930109439, 'random_state': 3272}.
[I 2019-05-18 07:02:10,871] Finished trial#57 resulted in value: 0.0019681361801170663. Current best value is 6.240043953774098e-06 with parameters: {'n_estimators': 904, 'max_depth': 8, 'max_features': 0.9905947830545193, 'min_samples_leaf': 0.4999748930109439, 'random_state': 3272}.
[I 2019-05-18 07:02:16,798] Finished trial#58 resulted in value: 0.00040390789199020805. Current best value is 6.240043953774098e-06 with parameters: {'n_estimators': 904, 'max_depth': 8, 'max_features': 0.9905947830545193, 'min_samples_leaf': 0.4999748930109439, 'random_state': 3272}.
[I 2019-05-18 07:02:35,966] Finished trial#59 resulted in value: 0.0013854286796257534. Current best value is 6.240043953774098e-06 with par

[I 2019-05-18 07:10:27,656] Finished trial#112 resulted in value: 0.0009879453553031537. Current best value is 1.2919228737593711e-06 with parameters: {'n_estimators': 741, 'max_depth': 6, 'max_features': 0.8675190121637301, 'min_samples_leaf': 0.4810693206559361, 'random_state': 2555}.
[I 2019-05-18 07:10:35,177] Finished trial#113 resulted in value: 0.00012459046767619727. Current best value is 1.2919228737593711e-06 with parameters: {'n_estimators': 741, 'max_depth': 6, 'max_features': 0.8675190121637301, 'min_samples_leaf': 0.4810693206559361, 'random_state': 2555}.
[I 2019-05-18 07:10:46,880] Finished trial#114 resulted in value: 0.0008633487749083354. Current best value is 1.2919228737593711e-06 with parameters: {'n_estimators': 741, 'max_depth': 6, 'max_features': 0.8675190121637301, 'min_samples_leaf': 0.4810693206559361, 'random_state': 2555}.
[I 2019-05-18 07:10:57,064] Finished trial#115 resulted in value: 0.0008141731244302816. Current best value is 1.2919228737593711e-06 w

[I 2019-05-18 07:18:46,694] Finished trial#168 resulted in value: 0.0015477238207405992. Current best value is 1.2919228737593711e-06 with parameters: {'n_estimators': 741, 'max_depth': 6, 'max_features': 0.8675190121637301, 'min_samples_leaf': 0.4810693206559361, 'random_state': 2555}.
[I 2019-05-18 07:19:02,487] Finished trial#169 resulted in value: 0.0025734006778688925. Current best value is 1.2919228737593711e-06 with parameters: {'n_estimators': 741, 'max_depth': 6, 'max_features': 0.8675190121637301, 'min_samples_leaf': 0.4810693206559361, 'random_state': 2555}.
[I 2019-05-18 07:19:09,094] Finished trial#170 resulted in value: 0.0002568009986588781. Current best value is 1.2919228737593711e-06 with parameters: {'n_estimators': 741, 'max_depth': 6, 'max_features': 0.8675190121637301, 'min_samples_leaf': 0.4810693206559361, 'random_state': 2555}.
[I 2019-05-18 07:19:23,741] Finished trial#171 resulted in value: 0.0014046273175260834. Current best value is 1.2919228737593711e-06 wi

In [33]:
for trial_i in mytrial:
    db.insert(trial_i)

In [34]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 31 by stratified')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
348,2019-05-18 07:10:17.473837,tune 31 by stratified,26,2.094741,0.000003,2.096300,0.000022,0.001559
298,2019-05-18 07:02:50.899293,tune 31 by stratified,26,2.098361,0.000004,2.099842,0.000018,0.001480
401,2019-05-18 07:18:09.249450,tune 31 by stratified,26,2.099309,0.000011,2.100723,0.000026,0.001414
409,2019-05-18 07:19:49.326856,tune 31 by stratified,26,2.102819,0.000003,2.103971,0.000020,0.001152
248,2019-05-18 06:54:38.989644,tune 31 by stratified,26,2.102814,0.000001,2.104052,0.000026,0.001237
261,2019-05-18 06:57:09.715539,tune 31 by stratified,26,2.111951,0.000003,2.112943,0.000015,0.000992
406,2019-05-18 07:19:02.467774,tune 31 by stratified,26,2.115084,0.000019,2.116300,0.000027,0.001216
270,2019-05-18 06:58:54.716802,tune 31 by stratified,26,2.115968,0.000008,2.117067,0.000016,0.001098
345,2019-05-18 07:09:37.742942,tune 31 by stratified,26,2.117832,0.000012,2.118752,0.000025,0.000920
377,2019-05-18 07:14:48.702307,tune 31 by stratified,26,2.117563,0.000019,2.118771,0.000022,0.001208


In [35]:
db.commit()

In [36]:
param = copy.deepcopy(df_trial.loc[348]['param'])
param['kfold']['type'] = 'group'
# run one try
mytrial=[]
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='remodel 348 use group')

In [37]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [38]:
df_trial.tail(1)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
437,2019-05-18 07:30:23.386685,26,2.09126,0.002742,2.103574,0.011848,0.012313


In [39]:
db.commit()